### Turing Machine Two for One

csc427: Theory of Automata and Complexity. 
<br>
university of miami
<br>
spring 2021.
<br>
Burton Rosenberg.
<br>
<br>
created: 10 April 2020
<br>last update: 16 April 2021

***


### Simulate Two Tapes on One Tape

Following the suggestion of Sipser, we simulate a multitape machine by taking the k tapes and putting them on one tape, 
<div style="padding:01.5em 5em;"><code>&dashv; &amp; T1 &amp; T2 &amp; ... &amp; Tk &amp;
</code></div>
Sipser suggests marking the locations of the k tape heads with a dot over the symbol. We will do it differently. For each original tape cell is a pair of cells, the first in the pair marks with an ! if the tape head is over the cell, and the second in the cell is the symbol itself. 

The construction has hints of a universal turing machine. The state transitions become "soft". Because we have to simulate the tape, we have to simulate the computation on the tape. We do this by gathering an encoding to the input of the transition function on a work area of the tape, to the right of everything else, and then entering a cascade of states that matches what was gathered, and then replaces it with an encoding of the output of the transition function.

The following construction is broken down into subroutines. Most of the subroutines have no dependency on the TM we are simultation. Only the state transition simulation step does. This net of states is hand built given the TM to simulate.

Furthermore, as much as possible state is pushed onto markings on the tape, so that the TM runs in a big loop, returning each time to the top of the loop with the tape markings the sole guide as to what will happen next. The loop is,

1. A preamble to get the tape into proper format.
2. A make space step that adds one blank space to the end of the two tapes, just in case it is needed.
3. A collection process copies the tape symbols to the work space of the tape, were the current state is also written.
4. A state transition process that replaces on the work space the input of the transition funciont with its output.
5. An update process that moves the tape heads and writes the new tape symbols.
6. Check if the new state is the accepting state. If so accept. Likewise the rejection state.
7. Otherwise return to step 2.



### TuringMachine class

In [1]:
import string
import sys
import os
import argparse
import re

#
# tm-sim.py
#
# author: bjr
# date: 21 mar 2020
# last update: 22 mar 2020
#    16 mar 2021, updated 
#     3 apr 2021, return conventions for accept/reject
#                 verbose_levels reimplemented
#                 character # is not allowed as a tape symbol
#                 for magical reasons, then " is also not allowed
#                 added class method help()
#                 
#
# copyright: Creative Commons. See http://www.cs.miami.edu/home/burt
#

# GRAMMAR for the TM description

# Comments (not shown in BNF) begin with a hash # and continue to the end
#    of the line
# The ident tokens are states
# The symbol tokens are tape symbolss
# The StateTransition semantics is:
#     tape_symbol_read tape_symbol_written action new_state
# The underscore _ is a tape blank:
# The : in a transition rule is the default tape symbol match when there is no
#    exactly matching transition rule; in the target section of the rule it 
#    is the value of the matchined tape symbol.

# A missing transition is considered a reject, not an error

class TuringMachine:
    
    verbose_levels = {'none':0,'verbose':1,'explain':2, 'debug':3}
    result_reasons = ['ok', 'transition missing', 'time limit']

    grammar = """
    M-> (Stanza [emptyline])*
    Stanza-> StartStanza | AcceptStanza | RejectStanza | StateStanze
    StartStanza-> "start" ":" ident
    AcceptStanza-> "accept" ":" ident ([newline] [indent] ident])*
    RejectStanza-> "reject" ":" ident ([newline] [indent] ident])*
    StateStanze-> "state" ":" ident ([newline] [indent] StateTransition)+
    StateTransition-> (symbol|special) (symbol|special) action ident
    action-> l|r|n|L|R|N
    symbol-> \w[!$-/]     # note: a tape symbol
    special-> ":"
    ident-> \w+           # note: name of a state

    """

    def __init__(self):
        self.start_state = "" # is an state identifier
        self.accept_states = set() # is a set of state identifiers
        self.reject_states = set() # is a set of state identifiers
        self.transitions = {} # is a map of (state,symbol):(state,symbol,action)
        self.current_state = "" 
        self.step_counter = 0
        self.all_actions = ["r","l","n"]
        self.tape = ['_']  # is a list of symbols
        self.position = 0
        self.verbose = 0
        self.result = 0

    def set_start_state(self,state):
        self.start_state = state

    def set_tape(self,tape_string):
        self.tape =  ['_' if symbol==':' or symbol==' ' else 
                          symbol for symbol in tape_string]

    def add_accept_state(self,state):
        self.accept_states.add(state)

    def add_reject_state(self,state):
        self.reject_states.add(state)
    
    def get_current_state(self):
        return self.curent_state

    def add_transition(self,state_from,read_symbol,
                       write_symbol,action,state_to):

        if action.lower() not in self.all_actions:
            # return something instead, nobody likes a chatty program
            return "WARNING: unrecognized action."
        x = (state_from, read_symbol)
        if x in self.transitions:
            # return something instead, nobody likes a chatty program
            return "WARNING: multiple outgoing states not allowed for DFA's."
        self.transitions[x] = (state_to,write_symbol,action)
        return None

    def restart(self,tape_string):
        self.current_state = self.start_state
        self.position = 0
        if len(tape_string)==0 :
            tape_string = '_'
        self.set_tape(tape_string)
        self.step_counter = 1

    def step_transition(self):
        c_s = self.current_state
        x = (c_s,self.tape[self.position])
        
        if x in self.transitions:
            (new_state, symbol, action ) = self.transitions[x]
        elif (c_s,':') in self.transitions:
            # wildcard code
            (new_state, symbol, action ) = self.transitions[(c_s,':')]
        else:
            # here we implement a rejection of convenience, if there is
            # no transition, tansition target is (:, n, A_REJECT_STATE)
            self.reason = 1
            print(f'no transition: {x}')
            return False
        
        # wildcard code
        if symbol==':':
            symbol = self.tape[self.position]

        self.current_state = new_state
        self.tape[self.position] = symbol

        shout = False
        if action.lower() != action:
            shout = True
            action = action.lower()
        
        if action == 'l' and self.position>0:
            self.position -= 1
        if action == 'r':
            self.position += 1
            if self.position==len(self.tape):
                self.tape[self.position:] = '_'
        if action == 'n':
            pass
   
        if shout or self.verbose == TuringMachine.verbose_levels['explain']:
            self.print_tape()
        if self.verbose == TuringMachine.verbose_levels['debug']:
            print("\t", self.step_counter, "\t", new_state, symbol, action)
            
        self.step_counter += 1
        return True

    def compute_tm(self,tape_string,step_limit=0,verbose='none'):
        self.verbose = TuringMachine.verbose_levels[verbose]
        self.result = 0
        self.restart(tape_string)
        if self.verbose == TuringMachine.verbose_levels[verbose]:
            self.print_tape()
        step = 0
            
        stop_states = self.accept_states.union(self.reject_states)
        while self.current_state not in stop_states:
            res = self.step_transition()
            if not res:
                # missing transition is considered a reject
                return False
            step += 1
            if step > step_limit:
                self.result = 2 
                return None
            
            if self.verbose == TuringMachine.verbose_levels['debug']:
                print(step, self.current_state, self.position, self.tape )

        if self.current_state in self.accept_states:
            return True
        return False

    def print_tape(self):
        t, p = self.tape, self.position
        s = ''.join(t[:p] + ['['] + [t[p]] + [']'] + t[p+1:])
        print(f'{self.current_state}:\t{s}')
    
    def print_tm(self):
        print("\nstart state:\n\t",self.start_state)
        print("accept states:\n\t",self.accept_states)
        print("reject states:\n\t",self.reject_states)
        print("transitions:")
        for t in self.transitions:
            print("\t",t,"->",self.transitions[t])
    
    @classmethod
    def help(cls):
        print('The verbose levels are:')
        for level in cls.verbose_levels:
            print(f'\t{cls.verbose_levels[level]}: {level}')
        print()
        print('The grammar for the Turing Machine description is:')
        print(cls.grammar)
        
        
### end class TuringMachine


class MachineParser:

    @staticmethod
    def turing(tm_obj, fa_string):
        """
        Code to parse a Turing Machine description into the Turing Machine object.
        """
        
        fa_array = fa_string.splitlines()
        line_no = 0 
        current_state = ""
        in_state_read = False
        in_accept_read = False
        in_reject_read = False

        for line in fa_array:
            while True:

                # comment lines are fully ignored
                if re.search('^\s*#',line):
                    break

                if re.search('^\s+',line):

                    if in_state_read:
                        m = re.search('\s+(\w|[!$-/:_])\s+(\w|[!$-/:_])\s+(\w)\s+(\w+)',line)
                        if m:
                            res = tm_obj.add_transition(current_state,
                                    m.group(1),m.group(2),m.group(3),m.group(4))
                            if res: 
                                print(res, f'line number {line_no}: {line}')
                                return False
                            break

                    if in_accept_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_accept_state(m.group(1))
                            break

                    if in_reject_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_reject_state(m.group(1))
                            break

                in_state_read = False
                in_accept_read = False
                in_reject_read = False

                # blank lines do end multiline input
                if re.search('^\s*$',line):
                    break ;

                m = re.search('^start:\s*(\w+)',line)
                if m:
                    tm_obj.set_start_state(m.group(1))
                    break

                m = re.search('^accept:\s*(\w+)',line)
                if m:
                    tm_obj.add_accept_state(m.group(1))
                    in_accept_read = True
                    break

                m = re.search('^reject:\s*(\w+)',line)
                if m:
                    tm_obj.add_reject_state(m.group(1))
                    in_reject_read = True
                    break

                m = re.search('^state:\s*(\w+)',line)
                if m:
                    in_state_read = True
                    current_state = m.group(1)
                    break

                print(line_no,"warning: unparsable line, dropping: ", line)
                return False
                break

            line_no += 1
        return True

### end class MachineParser



In [2]:

def create_and_test_turing_machine(tm_description, test_cases,verbose='none'):
    tm = TuringMachine()
    MachineParser.turing(tm,tm_description)
 
    print("\n*** TEST RUNS ***")

    for s in test_cases:
        # assume complexity is some quadratic
        res = tm.compute_tm(s,step_limit=10*(len(s)+5)**2,verbose=verbose)
        if res==True:
            print(f'ACCEPT input {s}\n')
        elif res==False:
            print(f'REJECT input {s}\n')
        else:
            print(f'ERROR on input {s}: {TuringMacine[tm.result]}')
            
    print("\n\n*** RUN COMPLETE ***\n\n")

# TuringMachine.help()


### Preamble

We use the tape area beyond the second tape to store the current state, and as a work space when simulating state transitions. The first step is to get the tape and head ready.


<div style="padding:01.5em 5em;"><code>PRECONDITION
    &dashv; [&amp;] T1 &amp; T2 &amp;
POSTCONDITION
    &dashv; $ T1 &amp; T2 &amp; QSTART [&sqcup;]
</code></div>

States as strings over 0, 1; and 0 is the start state.


In [3]:
TM_preamble = """# 

start: start
accept: A
reject: R

state: start
    : : N pr_enter
    
state: pr_enter
    & $ r in_01
    
state: in_01
    & : r in_02
    : : r in_01
    
state: in_02
    : : r in_02
    & : r in_04

state: in_04
    _ 0 r in_05

state: in_05
   : : N pr_exit
   
state: pr_exit
    : : N mo_enter

"""

### The Make Room Maneuver 


The tapes need to seem to be infinitely long to the left. Since they are placed on after the other, none of them (perhaps that last) can be infinitely long. Sipser adds blanks to the end of the tape on demand. In this implementation, we begin each round of state transition simulation with adding a blank to the end of each tape,

<div style="padding:01.5em 5em;"><code>PRECONDITION
    &dashv; &amp; T1 &amp; T2 &amp; ... &amp; Tk &amp;
POSTCONDITION
    &dashv; &amp; T1 [&sqcup;] &amp; T2 [&sqcup;] &amp; ... &amp; Tk [&sqcup;] &amp; 
</code></div>

Here is the big picture in the specific case of this simulation. Note that we will use the end of the tape as a work area. The name of the current state will always follow the second &amp;.

<div style="padding:01.5em 5em;">
<code>PRECONDITION:
    &dashv; T1 &amp; T2 &amp; QCURRENT [&sqcup;]
SHIFT from &amp; one left, &sqcup; fill vacated cell
    &dashv; T1 &amp; T2 &sqcup; &amp; QCURRENT [&sqcup;]
AND AGAIN 
    &dashv; T1 &amp; T2 &sqcup; &sqcup; &amp; QCURRENT [&sqcup;]
FIND LEFT END T2, AND SPECIAL MARK ITS RIGHT END
    &dashv; T1 [&amp;] T2 &sqcup; &sqcup; $ QCURRENT
SHIFT all up until $
    &dashv; T1 &amp; &sqcup; T2 &sqcup; [$] QCURRENT
RESTORE &amp; and find left edge again
    &dashv; T1 [&amp;] &sqcup; T2 &sqcup; &amp; QCURRENT
SHIFT all up until &amp;
    &dashv; T1 &amp; &sqcup; &sqcup; T2 [&amp;] QCURRENT
FIND &amp; ending T1 and more &amp; and get ready to repeat
    &dashv; T1 &sqcup; &sqcup; &amp; T2 &amp; QCURRENT [&sqcup;]
REPEAT THE DOUBLE SHIFT of above,
    &dashv; T1 &sqcup; &sqcup; &amp; T2 &sqcup; &sqcup; &amp; QCURRENT [&sqcup;]
</code>
</div>

The use of the dollar sign replacing &amp; is to reuse code. The shift left procedure runs twice. The dollar or ampersand is used in a branching decision whether to repeat the shift left procedure or to continue one. It would be similar to this python code,

<code>
    again = '$'
    while True:
        shift_left()
        if again == '$':
            again = '&'
        else:
            break
</code>

In [4]:

TM_make_space = """

# moves a blank across tape strings

state: mo_enter
    : : n mo_1_0

#  U&X[_] ->  U_&X[_], X is a 0,1 string

state: mo_1_0 # find &
    : : l mo_1_0
    & _ r mo_1_1
    
state: mo_1_1
    0 & r mo_1_2
    1 & r mo_1_3
    
state: mo_1_2
    0 0 r mo_1_2
    1 0 r mo_1_3
    _ 0 l mo_1_4 
    
state: mo_1_3
    0 1 r mo_1_2
    1 1 r mo_1_3
    _ 1 l mo_1_4

#  U&X[_] ->  U_&X[_], X is a 0,1 string

state: mo_1_4 # find &
    : : l mo_1_4
    & _ r mo_1_5
    
state: mo_1_5
    0 & r mo_1_6
    1 & r mo_1_7
    
state: mo_1_6
    0 0 r mo_1_6
    1 0 r mo_1_7
    _ 0 r mo_1_8
    
state: mo_1_7
    0 1 r mo_1_6
    1 1 r mo_1_7
    _ 1 r mo_1_8

state: mo_1_8 # tape is U__&X[_]
    : : l mo_2_0
    
state: mo_2_0 # find & 
    : : l mo_2_0
    & $ l mo_2_1
    
state: mo_2_1 # find &
    : : l mo_2_1
    & : r mo_2_2
    
state: mo_2_2  # shift characters
    0 _ r mo_2_3
    1 _ r mo_2_4
    ! _ r mo_2_5
    _ _ r mo_2_6
    
state: mo_2_3
    0 0 r mo_2_3
    1 0 r mo_2_4
    ! 0 r mo_2_5
    _ 0 r mo_2_6
    $ & l mo_2_1 # end first time through
    & : l mo_2_7 # end second time through
    
state: mo_2_4
    0 1 r mo_2_3
    1 1 r mo_2_4
    ! 1 r mo_2_5
    _ 1 r mo_2_6
    $ & l mo_2_1 # end first time through
    & : l mo_2_7 # end second time through
    
state: mo_2_5
    0 ! r mo_2_3
    1 ! r mo_2_4
    ! ! r mo_2_5
    _ ! r mo_2_6
    $ & l mo_2_1 # end first time through
    & : l mo_2_7 # end second time through
    
state: mo_2_6
    0 _ r mo_2_3
    1 _ r mo_2_4
    ! _ r mo_2_5
    _ _ r mo_2_6
    $ & l mo_2_1 # end first time through
    & : l mo_2_7 # end second time through
    
# U & _ _ V [v] & X -> U _ _ & V & X [_] 
state: mo_2_7 
    : : l mo_2_7
    & _ r mo_2_8
    
state: mo_2_8
    _ : r mo_2_9
    
state: mo_2_9
    _ & r mo_2_10
    
state: mo_2_10
    : : r mo_2_10
    & : r mo_2_11
    
state: mo_2_11
    : : r mo_2_11
    _ : N mo_2_end

# U__&V&X[_] -> U__&V__&X[_]

state: mo_2_end
    : : l mo_3_0

state: mo_3_0 # find &
    : : l mo_3_0
    & _ r mo_3_1
    
state: mo_3_1
    0 & r mo_3_2
    1 & r mo_3_3
    
state: mo_3_2
    0 0 r mo_3_2
    1 0 r mo_3_3
    _ 0 l mo_3_4
    
state: mo_3_3
    0 1 r mo_3_2
    1 1 r mo_3_3
    _ 1 l mo_3_4
    
#  U&X[_] ->  U_&X[_], X is a 0,1 string

state: mo_3_4 # find &
    : : l mo_3_4
    & _ r mo_3_5
    
state: mo_3_5
    0 & r mo_3_6
    1 & r mo_3_7
    
state: mo_3_6
    0 0 r mo_3_6
    1 0 r mo_3_7
    _ 0 r mo_3_8
    
state: mo_3_7
    0 1 r mo_3_6
    1 1 r mo_3_7
    _ 1 r mo_3_8

state: mo_3_8 # tape is U__&X[_]
    : : N mo_end
    
state: mo_end
    : : N co_enter
    

"""


### Collect the Two Tape Symbols

In preparation for the simulation of a state transition, we copy the two symbols under the tape heads to the rightmost region of the tape.


<div style="padding:01.5em 5em;">
<code>PRECONDITION:
    &dashv; T1 !x T1' &amp; T2 !y T2' &amp; QCURRENT [&sqcup;]
POSTCONDITION:
    &dashv; T1 !x T1' &amp; T2 !y T2' &amp; QCURRENT , y x [&sqcup;]
</code>
</div>

In [5]:
TM_collect_symbols = """

# collect the values under the two tape heads

state: co_enter
    : : n co_00_a
    
state: co_00_a
    _ $ l co_01_a
    
state: co_01_a
    & : L co_02_a
    : : L co_01_a
    
state: co_02_a
    0 : l co_02_a
    1 : l co_03_a
    _ : l co_04_a
    ! : r co_00_b
    
state: co_03_a
    0 : l co_02_a
    1 : l co_03_a
    _ : l co_04_a
    ! : r co_00_c

state: co_04_a
    0 : L co_02_a
    1 : l co_03_a
    _ : l co_04_a
    ! : r co_00_d

# the first head sees a 0

state: co_00_b
    : : r co_00_b
    $ , r co_01_b
    
state: co_01_b
    _ 0 r co_02_b
    
state: co_02_b
    _ $ l co_02_e

# the first head sees a 1

state: co_00_c
    : : r co_00_c
    $ , r co_01_c
    
state: co_01_c
    _ 1 r co_02_c
    
state: co_02_c
    _ $ l co_02_e

# the first head sees a _

state: co_00_d
    : : r co_00_d
    $ , r co_01_d
    
state: co_01_d
    _ - r co_02_b
    
state: co_02_d
    _ $ l co_02_e
 
# now get the second head.

state: co_02_e
    : : l co_02_e
    & : l co_03_e

state: co_03_e
    : : l co_03_e
    & : l co_02_f
    
state: co_02_f
    0 : l co_02_f
    1 : l co_03_f
    _ : l co_04_f
    ! : r co_00_g
    
state: co_03_f
    0 : l co_02_f
    1 : l co_03_f
    _ : l co_04_f
    ! : r co_00_h

state: co_04_f
    0 : L co_02_f
    1 : l co_03_f
    _ : l co_04_f
    ! : r co_00_j

# the second head sees a 0
state: co_00_g
    : : r co_00_g
    $ 0 r co_00_k

# the second head sees a 1
state: co_00_h
    : : r co_00_h
    $ 1 r co_00_k

# the second head sees a _
state: co_00_j
    : : r co_00_j
    $ - r co_00_k

state: co_00_k
    : : N co_exit
    
state: co_exit
    : : N st_enter
    

"""



### State Transition Simulation

We then replace the input to the state transition function, found at the right end of the tape, with the output of the state transition function.



<div style="padding:01.5em 5em;">
<code>PRECONDITION:
    &dashv; T1 !x T1' &amp; T2 !y T2' &amp; QCURRENT , y x [&sqcup;]
TRANSITION:
    &delta; ( QCURRENT, x, y ) &mapsto; ( QNEXT, d1, x', d2, y' )
POSTCONDITION:
    &dashv; T1 !x T1' &amp; T2 !y T2' &amp; QNEXT , d1 d2 y' x' [&sqcup;]
</code>
</div>


### Updating the Tapes: Symbol Write-down

The symbols to write-down, x and y, are at the right edge of the tape. Sense the
symbol and erase it, and navigate by finding either one or two ampersands's then the exclamation point. Seek to right by searching for either a comma or a dot.

The number of states is compressed by using a trick to reuse the core code for both write-down on tape 1 and tape 2. The comma in the work area is changed to a dot for the first time through. When the seek to right sees a dot, it changes it back to a comma but it triggers a second use of much of the code to change the second symbol.

The code block called Section A will be duplicated, one for each symbol 0 or 1 being written down. 

_Note: It might be possible to do this without holding the sitatutation in state by swapping the symbol to place location by location as the seek moves left._

#### First time through preamble

<div style="padding:01.5em 5em;">
<code>PRECONDITION:
    &dashv; T1 &amp; T2 &amp; QCURRENT , s t y x [&sqcup;]
POSTCONDITION:
    &dashv; T1 &amp; T2 [t] &amp; QNEXT . s t y 
GOTO SECTION A
</code>
</div>

#### First time through continued


<div style="padding:01.5em 5em;">
<code>SECTION A
PRECONDITION FIRST TIME 
    &dashv; T1 &amp; T2 [t] &amp; QNEXT . s u y 
FIND &amp; then FIND !
    &dashv; T1 ! [z] T1' &amp; T2 t &amp; QNEXT . s u y 
REPLACE WITH x AND FIND . or ,
    &dashv; T1 ! x T1' &amp; T2 t &amp; QNEXT [.] s u y 
REPLACE AND GET y
    &dashv; T1 ! x T1' &amp; T2 t &amp; QNEXT , s [u]
GOTO SECTION A
</code>
</div> 

#### Second time through

<div style="padding:01.5em 5em;">
<code>SECTION A
PRECONDITION SECOND TIME
    &dashv; T1 ! x T1' &amp; T2 t &amp; QNEXT , s [u]
FIND &amp; then FIND !
    &dashv; T1 ! x T1' &amp; T2 ! [r] T2' t &amp; QNEXT , s u
REPLACE WITH y AND FIND . or ,
    &dashv; T1 ! x T1' &amp; T2 ! y T2' t &amp; QNEXT [,] s u
DONE
</code>
</div>



### Updating the Tapes: Moving Heads

Two symbols, each L or R are at the right end of tape. Sense the symbol and erase it, and navigate by finding either on or two ampersands then the exclamation point. If R sensed, erase the exclamation point, move two cells right and write an exclamation point and seek right to dot or comma.

Otherwise if L sensed, erase the exclamation point and move one cell left. If ampersand or dollar sign sensed move one cell right, else move another cell right. Write an exclamation point and seek right to dot or comma.

The number of states is compressed by using a trick to reuse the core code for both head movements on tape 1 and tape 2. The comma in the work area is changed to a dot for the first time through. When the seek to right sees a dot, it changes it back to a comma but it triggers a second use of much of the code to move the second head.

The code block called Section B and Section C differ in where a move right or a move left is being processed. It is the continuation of a preamble affixed to the first time through that allows the code to repeat one time, for the second time through.


#### First time through preamble: RIGHT


<div style="padding:01.5em 5em;">
<code>PRECONDITION:
    &dashv; T1 &amp; T2 &amp; QCURRENT , s R [&sqcup;]
POSTCONDITION:
    &dashv; T1 &amp; T2 [t] &amp; QNEXT . s
GOTO SECTION B
</code>
</div>


#### First time through preamble: LEFT


<div style="padding:01.5em 5em;">
<code>PRECONDITION:
    &dashv; T1 &amp; T2 &amp; QCURRENT , s L [&sqcup;]
POSTCONDITION:
    &dashv; T1 &amp; T2 [t] &amp; QNEXT . s
GOTO SECTION C
</code>
</div>


#### First time through continued: RIGHT


<div style="padding:01.5em 5em;">
<code>SECTION B
PRECONDITION FRIST TIME:
    &dashv; T1 &amp; T2 [t] &amp; QNEXT . s
AFTER FIND &amp; then find !:
    &dashv; T1 [!] u &sqcup; T1' &amp; T2 t &amp; QNEXT . s 
AFTER REPLACEMENTS AND FIND TO , OR .
    &dashv; T1  &sqcup; u ! T1' &amp; T2 t &amp; QNEXT [.] s 
REPLACE . AND GET s
    &dashv; T1  &sqcup; u ! T1' &amp; T2 t &amp; QNEXT , [s]
GOTO SECTION B OR SECTION C
</code>
</div>


#### Second time through: RIGHT


<div style="padding:01.5em 5em;">
<code>SECTION B
PRECONDITION SECOND TIME: 
    &dashv; T1  &sqcup; u ! T1' &amp; T2 &amp; QNEXT , [R]
AFTER FIND &amp; then find !:
    &dashv; T1  &sqcup; u ! T1' &amp; T2 [!] v &sqcup; T2' &amp; QNEXT ,
AFTER REPLACEMENTS AND FIND TO , OR .
    &dashv; T1  &sqcup; u ! T1' &amp; T2 &sqcup; u ! T2' &amp; QNEXT [,]
</code>
</div>
    

#### First time through continued: LEFT


<div style="padding:01.5em 5em;">
<code>SECTION C
PRECONDITION FRIST TIME:
    &dashv; T1 &amp; T2 [t] &amp; QNEXT . s
AFTER FIND &amp; then find !:
    &dashv; T1 u [!] T1' &amp; T2 t &amp; QNEXT . s 
BLANK WRITTEN, SENSE u 
    &dashv; T1 [u] &sqcup; T1' &amp; T2 t &amp; QNEXT . s 
    IF u is &amp; or $, STEP RIGHT
        &dashv; T1 u [&sqcup;] T1' &amp; T2 t &amp; QNEXT . s 
    ELSE STEP LEFT
        &dashv; T1'' [&sqcup;] u &sqcup; T1' &amp; T2 t &amp; QNEXT . s 
WRITE ! SEEK RIGHT , OR .
    &dashv; T1''' ! &amp; T2 t &amp; QNEXT [.] s 
REPLACE . AND GET s
    &dashv; T1  &sqcup; u ! &amp; T2 t &amp; QNEXT , [s]
GOTO SECTION B OR SECTION C
</code>
</div>


#### Second time through: LEFT


<div style="padding:01.5em 5em;">
<code>SECTION C
PRECONDITION SECOND TIME: 
    &dashv; T1  &amp; T2 &amp; QNEXT , [L]
AFTER FIND &amp; then find !:
    &dashv; T1 &amp; T2' w [!] u &sqcup; T2' &amp; QNEXT ,
WRITE BLANK SENSE w
    &dashv; T1 &amp; T2' [w] &sqcup; u &sqcup; T2' &amp; QNEXT ,
    IF w an &amp; or $, STEP RIGHT
    &dashv; T1 &amp; T2' w [&sqcup;] u &sqcup; T2' &amp; QNEXT ,
    ELSE STEP LEFT
    &dashv; T1 &amp; T2'' [&sqcup;] w &sqcup; u &sqcup; T2' &amp; QNEXT ,
WRITE ! AND SEEK RIGHT TO DOT OR COMMA
    &dashv; T1 &amp; T2' ! T2' &amp; QNEXT [,]
DONE
</code>
</div>
    




### Machine Termination Check

Under construction. 


In [6]:
TM_under_construction = """

state: st_enter
    : : N st_exit
    
state: st_exit
    : : N the_rest
    
state: the_rest
    : : n A 
"""

In [7]:

TM = TM_preamble + TM_make_space + TM_collect_symbols + TM_under_construction

TM_test = [
    "&!0 0 0&!1 1 1&",
    "&! &! &",
    "& 0!1 0& 1!0 1&",
    "& 0!1& 1!1&",
]

create_and_test_turing_machine(TM,TM_test,verbose='explain')  
    


*** TEST RUNS ***
start:	[&]!0_0_0&!1_1_1&
pr_enter:	[&]!0_0_0&!1_1_1&
in_01:	$[!]0_0_0&!1_1_1&
in_01:	$![0]_0_0&!1_1_1&
in_01:	$!0[_]0_0&!1_1_1&
in_01:	$!0_[0]_0&!1_1_1&
in_01:	$!0_0[_]0&!1_1_1&
in_01:	$!0_0_[0]&!1_1_1&
in_01:	$!0_0_0[&]!1_1_1&
in_02:	$!0_0_0&[!]1_1_1&
in_02:	$!0_0_0&![1]_1_1&
in_02:	$!0_0_0&!1[_]1_1&
in_02:	$!0_0_0&!1_[1]_1&
in_02:	$!0_0_0&!1_1[_]1&
in_02:	$!0_0_0&!1_1_[1]&
in_02:	$!0_0_0&!1_1_1[&]
in_04:	$!0_0_0&!1_1_1&[_]
in_05:	$!0_0_0&!1_1_1&0[_]
pr_exit:	$!0_0_0&!1_1_1&0[_]
mo_enter:	$!0_0_0&!1_1_1&0[_]
mo_1_0:	$!0_0_0&!1_1_1&0[_]
mo_1_0:	$!0_0_0&!1_1_1&[0]_
mo_1_0:	$!0_0_0&!1_1_1[&]0_
mo_1_1:	$!0_0_0&!1_1_1_[0]_
mo_1_2:	$!0_0_0&!1_1_1_&[_]
mo_1_4:	$!0_0_0&!1_1_1_[&]0
mo_1_5:	$!0_0_0&!1_1_1__[0]
mo_1_6:	$!0_0_0&!1_1_1__&[_]
mo_1_8:	$!0_0_0&!1_1_1__&0[_]
mo_2_0:	$!0_0_0&!1_1_1__&[0]_
mo_2_0:	$!0_0_0&!1_1_1__[&]0_
mo_2_1:	$!0_0_0&!1_1_1_[_]$0_
mo_2_1:	$!0_0_0&!1_1_1[_]_$0_
mo_2_1:	$!0_0_0&!1_1_[1]__$0_
mo_2_1:	$!0_0_0&!1_1[_]1__$0_
mo_2_1:	$!0_0_0&!1_[1]_1__$0_
m